<a href="https://colab.research.google.com/github/AndreAmaduzzi/tensorflow-vs-pytorch/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch implementation of Object Detection Network

## Import Dependencies

In [36]:
from __future__ import print_function
from __future__ import division

import copy
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import matplotlib.pyplot as plt
import numpy as np
import random
import requests
import os
import time

from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from timeit import default_timer as timer
from typing import Callable, Dict, List, Tuple, Union
from torchvision import datasets, models, transforms
from torchsummary import summary
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

## Runtime Settings

In [37]:
device = "cpu"
if torch.cuda.is_available:
  print('All good, a Gpu is available')
  device = torch.device("cuda:0")  
else:
  print('Please set GPU via Edit -> Notebook Settings.')

All good, a Gpu is available


## Reproducibility and Deterministic Mode

In [38]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use. 
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(seed=7)

## Building Dataset
Dataset: [BCCD dataset](https://public.roboflow.com/object-detection/bccd)

3 classes (white blood cells, red blood cells, platelets)

364 images

In [55]:
# TODO: change dataset, for simplicity
from torchvision.datasets import OxfordIIITPet

In [40]:
# Setting data loading parameters
bs = 28
n_workers = 2
img_transformations = transforms.ToTensor()   # transformations to apply to the input images

In [41]:
trainval_ds = OxfordIIITPet(root='./dataset/trainval/',
                         split='trainval',
                         download=True,
                         transform=img_transformations)



The function passed as [```collate_fn```](https://pytorch.org/docs/stable/data.html) argument is used to collate lists of samples into batches when they have different size. This function is the function that processes the batch you want to return from your `DataLoader`.

In [42]:
def my_collate(batch):
    return tuple(batch)

In [53]:
trainval_dl = DataLoader(dataset=trainval_ds,
                      batch_size=bs,
                      shuffle=True,
                      num_workers = n_workers,
                      collate_fn=my_collate
                      )

for i, data in enumerate(trainval_dl):
  print(data[0][0].shape)
  print(data[0][1])
  if i>1:
    break

torch.Size([3, 333, 500])
19
torch.Size([3, 300, 216])
2
torch.Size([3, 375, 500])
34
